# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.multiclass import type_of_target
import numpy as np





nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eart_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eart_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eart_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_message', con=engine)
X = df['message'].values
Y = df.drop(['id','message','original','genre'],axis=1)
category_names = Y.columns
Y = Y.values

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
Y[500]

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    text = text.strip().lower().translate(str.maketrans('', '', string.punctuation))

    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmed

print(tokenize('Weather update - a cold front from Cuba that could pass over Haiti'))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001E30765F1E0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)

In [20]:
#type_of_target(y_test)
print(classification_report(y_test, y_pred,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.85      0.93      0.89      4046
               request       0.86      0.50      0.64       917
                 offer       0.00      0.00      0.00        23
           aid_related       0.81      0.54      0.65      2202
          medical_help       0.58      0.03      0.05       424
      medical_products       0.65      0.04      0.08       264
     search_and_rescue       0.56      0.03      0.06       159
              security       0.00      0.00      0.00        99
              military       0.58      0.04      0.07       176
                 water       0.90      0.26      0.41       336
                  food       0.87      0.45      0.59       581
               shelter       0.87      0.21      0.34       442
              clothing       0.77      0.11      0.20        88
                 money       0.83      0.04      0.08       125
        missing_people       0.00      

C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=50))
])

In [24]:
parameters = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [5, 7]
}
cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
parameters = parameters = {
    'tfidf__use_idf': [True, False]
    'clf__n_estimators': [50, 100],
    
    
}

In [ ]:
cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

In [ ]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=columns))

### 9. Export your model as a pickle file

In [ ]:
with open("../models/disaster_response_model.pkl", 'wb') as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.